In [77]:
import torch
import torchvision
import torchvision.models.segmentation.deeplabv3 as dlv3
import torchvision.transforms.functional as tf
import numpy as np
import data
import utils
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [78]:
def show_prediction(input_image, pred_image, label_color_map, label_string_map):
    label_legend_col = []

    
    # for i in range(len(label_color_map)):
    #     color = label_color_map.values()[i] + [255]
    #     color = np.array(color, dtype=np.float32)
    #     color /= 255.0
    #     label_legend_col.append(color.tolist())

def map_scalars_to_colors(scalars_mask, pixel_mapping):
    colors_mask = Image.new(mode="RGB", size=(scalars_mask.size[0], scalars_mask.size[1]))
    for i in range(scalars_mask.size[0]):
        for j in range(scalars_mask.size[1]):
            p =  scalars_mask.getpixel((i,j))
            colors_mask.putpixel((i,j), pixel_mapping[scalars_mask.getpixel((i,j))])
    return colors_mask

In [79]:
model_parameters_load_path = r'G:\My Drive\Master IVA\Master Thesis\Models\20230407_013726\deeplabv3_model.pt'
    
ipt_path = r'G:\My Drive\Master IVA\Master Thesis\Datasets\real\test\rgb\real_rgb_3.png'
tgt_path = r'G:\My Drive\Master IVA\Master Thesis\Datasets\synthetic\train\semantic_segmentation_mapped\synthetic_semantic_segmentation_1000.png'

num_classes = len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys())
infer_width_resize = 512
infer_height_resize = 256

In [80]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dlv3.deeplabv3_resnet50(pretrained=False, progress=True, num_classes=17)
model.load_state_dict(torch.load(model_parameters_load_path))
model.eval()
model.to(device)
print('Loaded model to device.')

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loaded model to device.


In [81]:
pil_ipt = Image.open(f'{ipt_path}', mode='r')
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((infer_height_resize, infer_width_resize)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.0)])
ipt = tf.to_tensor(np.array(pil_ipt).astype(np.float32))
ipt = transform(ipt)

pil_tgt = Image.open(f'{tgt_path}', mode='r',)
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((infer_height_resize, infer_width_resize)),
                                            torchvision.transforms.RandomHorizontalFlip(p=0.0)])

tgt = Image.open(tgt_path, 'r',)
tgt = np.array(tgt).astype(np.int_)
tgt = tf.to_tensor(tgt)


In [82]:
# tgt_colors_mask = map_scalars_to_colors(tgt, data.SemanticLabelMapper.ID_TO_COLOR['common'])

In [83]:
# from matplotlib.patches import Rectangle
# from matplotlib.gridspec import GridSpec

# rgb_tgt_blend = Image.blend(ipt, tgt_colors_mask, 0.5)

# label_rectangles = []
# for c in data.SemanticLabelMapper.ID_TO_COLOR['common'].values():
#     c_normalized = [c_val / 255.0 for c_val in c]
#     label_rectangles += [Rectangle((0,0), width=1, height=1, color=tuple(c_normalized))]

# label_names = [v for v in data.SemanticLabelMapper.ID_TO_STRING['common'].values()]

# fig, ax = plt.subplots(1, 2,  dpi=300, gridspec_kw={'width_ratios': [10, 1]}, figsize=(10,5))
# ax[0].imshow(np.array(rgb_tgt_blend), interpolation='nearest')
# ax[1].legend(label_rectangles,label_names, ncol=1, fontsize=11, loc='center')
# ax[1].axis('off')
# ax[0].axis('off')
# fig.tight_layout()
# # fig.show()

In [85]:
# ignore_label = 19
with torch.no_grad():
    
    confusion_mat = utils.compute_confusion_matrix(predictions=tgt, targets=tgt, num_classes=len(data.SemanticLabelMapper.ID_TO_COLOR['common'].keys()))
    miou, ious = utils.measure_performance(confusion_mat=confusion_mat, num_classes=len(data.SemanticLabelMapper.ID_TO_COLOR['common'].keys()), ignore_label=None)

    print(f'Mean IoU: {miou}')
    print(f'IoUs: {ious}')
    

KeyboardInterrupt: 